### Imports and folder paths

In [1]:
!pip install numpy
!pip install opencv-python
!pip install matplotlib
import numpy as np
import cv2 as cv
import os
#import glob
import matplotlib.pyplot as plt
#from numpy.random import uniform
#import pdb

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
my_cwd_path = './'
train_path = './antrenare/'
test_path = './testare/'
aux_path = './imagini_auxiliare/'
template_path = './templates/'

## Image functions

In [3]:
def color_filter(img, filter_type):
    
    if filter_type == 'full_blue':
        low_group = (15, 0, 144)
        high_group = (255, 255, 255)
        img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
        mask_group_hsv = cv.inRange(img_hsv, low_group, high_group)

        res = cv.bitwise_and(img, img, mask=mask_group_hsv)   
        
    elif filter_type == 'dark_table':
        low_group = (15, 0, 70)
        high_group = (255, 128, 255)
        img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
        mask_group_hsv = cv.inRange(img_hsv, low_group, high_group)
        res = cv.bitwise_and(img, img, mask=mask_group_hsv) 

    elif filter_type == 'dark_table_2':
        low_group = (12, 0, 0)
        high_group = (122, 122, 255)
        img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
        mask_group_hsv = cv.inRange(img_hsv, low_group, high_group)
        res = cv.bitwise_and(img, img, mask=mask_group_hsv) 

    elif filter_type == 'corner_points':
        low_group = (15, 0, 0)
        high_group = (100, 100, 255)
        img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
        mask_group_hsv = cv.inRange(img_hsv, low_group, high_group)
        res = cv.bitwise_and(img, img, mask=mask_group_hsv)
    
    elif filter_type == 'boost_filter':
        low_group = (15, 90, 0)
        high_group = (255, 255, 255)
        img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
        mask_group_hsv = cv.inRange(img_hsv, low_group, high_group)
        res = cv.bitwise_and(img, img, mask=mask_group_hsv) 

    elif filter_type == 'empty_filter':
        low_group = (92, 25, 190)
        high_group = (111, 80, 255)
        img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
        mask_group_hsv = cv.inRange(img_hsv, low_group, high_group)
        res = cv.bitwise_and(img, img, mask=mask_group_hsv) 

    return mask_group_hsv, res

In [4]:
def show_image(title,image):
    image=cv.resize(image,(0,0),fx=0.3,fy=0.3)
    print(title)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [5]:
def extrage_careu(image, filter_name, corner_points_l=None):
    original_image = image.copy()
    
    corner_points_copy = corner_points_l
    corner_points_l = None
    
    #show_image('original_image', original_image)
    mask,image = color_filter(image, 'corner_points')
    #show_image('image_color_filter', image)
    
    
    image = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    image_m_blur = cv.medianBlur(image,3)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5)
    image_sharpened = cv.addWeighted(image_m_blur, 1.2, image_g_blur, -0.8, 0)
    #show_image('image_sharpened',image_sharpened)
    _, thresh = cv.threshold(image_sharpened, 30, 255, cv.THRESH_BINARY)

    kernel = np.ones((5, 5), np.uint8)
    thresh = cv.dilate(thresh, kernel)
    thresh = cv.dilate(thresh, kernel)
    thresh = cv.dilate(thresh, kernel)
    thresh = cv.erode(thresh, kernel)
    #show_image('image_thresholded',thresh)

    edges =  cv.Canny(thresh ,20,100)
    #show_image('edges',edges)
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0

    
    possible_corners = []  

    for i in range(len(contours)):
        if len(contours[i]) > 3:  
            possible_top_left = None
            possible_bottom_right = None
        
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point
                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1]:
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis=1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]

            quadrilateral = np.array(
                [possible_top_left, possible_top_right, possible_bottom_right, possible_bottom_left]
            )
            area = cv.contourArea(quadrilateral)

            if area > 100000:  
                possible_corners.append((area, quadrilateral))

    possible_corners = sorted(possible_corners, key=lambda x: x[0], reverse=True)

    for area, quadrilateral in possible_corners:
        #print(f"Area: {area}, Quadrilateral: {quadrilateral}")
        top_left, top_right, bottom_right, bottom_left = quadrilateral
    if corner_points_l is not None:
        top_left, top_right, bottom_right, bottom_left = corner_points_l
     
    #return image

    width = 1400 #810
    height = 1400 #810
    
    mask,original_image = color_filter(original_image, filter_name)
    original_image = cv.cvtColor(original_image,cv.COLOR_BGR2GRAY)
    
    image_copy = cv.cvtColor(original_image.copy(),cv.COLOR_GRAY2BGR)
    #image_copy = original_image.copy()
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    #show_image("detected corners",image_copy)

    mask_copy = cv.cvtColor(mask.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    #show_image("mask detected corners",mask_copy)

    
    puzzle = np.array([top_left, top_right, bottom_right, bottom_left], dtype="float32")
    destination = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype="float32")

    M = cv.getPerspectiveTransform(puzzle, destination)
    result_mask = cv.warpPerspective(mask_copy, M, (width, height))
    result = cv.warpPerspective(image_copy, M, (width, height))
    
    corner_points_l = np.array([top_left, top_right, bottom_right, bottom_left])
    
    return result, result_mask, corner_points_l

### Corner_points test

In [6]:
# img = cv.imread(train_path+'1_01.jpg')
# #mask,img = color_filter(img, 'dark_table_2')
# #show_image('image_color_filter', img)
# result,_,corner_points=extrage_careu(img, 'corner_points')
# #show_image('img',result)


### Grid template

In [7]:
lines_horizontal=[]
for i in range(0,1400+1,100):
    l=[]
    l.append((0,i))
    l.append((1400-1,i))
    lines_horizontal.append(l)

In [8]:
lines_vertical=[]
for i in range(0,1400+1,100):
    l=[]
    l.append((i,0))
    l.append((i,1400-1))
    lines_vertical.append(l)

### Testing the grid template

In [9]:
# img = cv.imread(train_path+'1_04.jpg')
# #mask,img = color_filter(img, 'dark_table_2')
# result,result_mask,_ = extrage_careu(img, 'dark_table_2')
# #show_image('img',result)
# for line in  lines_vertical :
#     cv.line(result, line[0], line[1], (0, 255, 0), 5)
#     for line in  lines_horizontal :
#         cv.line(result, line[0], line[1], (0, 0, 255), 5)
# #show_image('img',result)



In [10]:

# empty_board = cv.imread(aux_path+'02.jpg')
# result_empty_board,result_mask_empty_board,_ = extrage_careu(empty_board, 'empty_filter')
# #show_image('empty_board',result_empty_board)
# for line in  lines_vertical :
#     cv.line(result_empty_board, line[0], line[1], (0, 255, 0), 5)
#     for line in  lines_horizontal :
#         cv.line(result_empty_board, line[0], line[1], (0, 0, 255), 5)
# #show_image('empty_board',result_empty_board)
# result_empty_board_gray = cv.cvtColor(result_empty_board, cv.COLOR_BGR2GRAY)
# #show_image('empty_board',result_empty_board_gray)



In [11]:
# #show_image('empty_board',result_mask_empty_board)
# for line in  lines_vertical :
#     cv.line(result_mask_empty_board, line[0], line[1], (0, 255, 0), 5)
#     for line in  lines_horizontal :
#         cv.line(result_mask_empty_board, line[0], line[1], (0, 0, 255), 5)
# #show_image('empty_board',result_mask_empty_board)
# result_mask_empty_board_gray = cv.cvtColor(result_mask_empty_board, cv.COLOR_BGR2GRAY)
# #show_image('empty_board',result_mask_empty_board_gray)

### Functions for ox si fullboard configurations

In [12]:
# def determina_configuratie_careu_ox(thresh,lines_horizontal,lines_vertical):
#     matrix = np.empty((14,14), dtype='str')
#     for i in range(len(lines_horizontal)-1):
#         for j in range(len(lines_vertical)-1):
#             y_min = lines_vertical[j][0][0]
#             y_max = lines_vertical[j + 1][1][0]
#             x_min = lines_horizontal[i][0][1]
#             x_max = lines_horizontal[i + 1][1][1]
#             patch = thresh[x_min+10:x_max-10, y_min+10:y_max-10].copy()
#             #completati codul aici
#             medie = np.mean(patch)
#             #print('Patch:', patch)
#             #print('Medie:', medie)
#             if medie > 30:
#               matrix[i][j] = 'x'
#               #print('Patch:', patch)
#               #print('Medie:', medie)
#             else:
#               matrix[i][j] = 'o'

#     return matrix

In [13]:
def vizualizare_configuratie(result,matrix,lines_horizontal,lines_vertical):
    #print(result)
    for i in range(len(lines_horizontal) - 1):
        for j in range(len(lines_vertical) - 1):
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1]
            x_max = lines_horizontal[i + 1][1][1]
            if matrix[i][j] == 'x':
                cv.rectangle(result, (y_min, x_min), (y_max, x_max), color=(255, 0, 0), thickness=5)

In [14]:
def determina_configuratie_careu_ox_fullboard(thresh,lines_horizontal,lines_vertical):
    matrix = np.empty((14,14), dtype='str')
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1]
            x_max = lines_horizontal[i + 1][1][1]
            patch = thresh[x_min+15:x_max-15, y_min+15:y_max-15].copy()
            #completati codul aici
            medie = np.mean(patch)
            #print('Patch:', patch)
            #print('Medie:', medie)
            if medie > 120 and medie < 245:
              matrix[i][j] = 'x'
              #print('Patch:', patch)
              #print('Medie:', medie)
            else:
              matrix[i][j] = 'o'

    return matrix

### Small test

In [15]:
# files = sorted(os.listdir(train_path))
# prev_board = None
# ok_counter = 5
# for file in files:
#   if file[-3:]=='jpg':
#     img = cv.imread(train_path+file)
#     result,result_mask,_ = extrage_careu(img, 'dark_table_2', corner_points)
#     show_image('result'+file, result)
#     _, result = cv.threshold(result, 90, 255, cv.THRESH_BINARY)
#     #_, result = cv.threshold(result, 90, 255, cv.THRESH_BINARY)
#     #_, result = cv.threshold(result, 90, 255, cv.THRESH_BINARY)
#     show_image('result=thresh'+file, result)

#     matrice_fullboard = determina_configuratie_careu_ox_fullboard(result, lines_horizontal, lines_vertical)
#     vizualizare_configuratie(result, matrice_fullboard, lines_horizontal, lines_vertical)
#     print(matrice_fullboard)
#     show_image('result_diff'+file, result)
    
#     ok_counter -= 1
#     if ok_counter < 0:
#         break

## Board

In [16]:
# game_board = np.empty((14,14), dtype='str')
# game_special_board = np.empty((14,14), dtype='str')

# for i in range(len(lines_horizontal)-1):
#     for j in range(len(lines_vertical)-1):
#         game_special_board[i][j] = '#'

# game_special_board[3][6] = '+'
# game_special_board[4][7] = '+'
# game_special_board[9][6] = '+'
# game_special_board[10][7] = '+'
# game_special_board[6][4] = '+'
# game_special_board[6][10] = '+'
# game_special_board[7][3] = '+'
# game_special_board[7][9] = '+'

# game_special_board[4][6] = '*'
# game_special_board[3][7] = '*'
# game_special_board[10][6] = '*'
# game_special_board[9][7] = '*'
# game_special_board[6][3] = '*'
# game_special_board[6][9] = '*'
# game_special_board[7][4] = '*'
# game_special_board[7][10] = '*'
        
# for i in range(len(lines_horizontal)-1):
#     for j in range(len(lines_vertical)-1):
#         if i in [0,6,7,13] and j in [0,6,7,13]:
#             game_special_board[i][j] = 't'
#         if i in [1,4,9,12] and j in [1,4,9,12] and (i+3 == j or i == j+3 or i+3 == 13-j or i-3 == 13-j):
#             game_special_board[i][j] = '/'
#         if i in [2,5,8,11] and j in [2,5,8,11] and (i+3 == j or i == j+3 or i+3 == 13-j or i-3 == 13-j) and i != j and i != 13-j:
#             game_special_board[i][j] = '-'
#         if i in [1,2,3,4,9,10,11,12] and j in [1,2,3,4,9,10,11,12] and (i == j or i == 13-j):
#             game_special_board[i][j] = 'd'
            
# game_special_board[6][6] = '1'
# game_special_board[6][7] = '2'
# game_special_board[7][6] = '3'
# game_special_board[7][7] = '4'
# print(game_special_board)

# game_board[6][6] = '1'
# game_board[6][7] = '2'
# game_board[7][6] = '3'
# game_board[7][7] = '4'

# lista_cifre = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,24,25,27,28,30,32,35,36,40,42,45,48,49,50,54,56,60,63,64,70,72,80,81,90]
# coordonate_board = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N']

In [17]:
def clear_board(game_board = None, game_special_board = None):
    game_board = np.empty((14,14))
    game_special_board = np.empty((14,14), dtype='str')

    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            game_special_board[i][j] = '#'
            game_board[i][j] = '-1'

    game_special_board[3][6] = '+'
    game_special_board[4][7] = '+'
    game_special_board[9][6] = '+'
    game_special_board[10][7] = '+'
    game_special_board[6][4] = '+'
    game_special_board[6][10] = '+'
    game_special_board[7][3] = '+'
    game_special_board[7][9] = '+'

    game_special_board[4][6] = '*'
    game_special_board[3][7] = '*'
    game_special_board[10][6] = '*'
    game_special_board[9][7] = '*'
    game_special_board[6][3] = '*'
    game_special_board[6][9] = '*'
    game_special_board[7][4] = '*'
    game_special_board[7][10] = '*'
        
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            if i in [0,6,7,13] and j in [0,6,7,13]:
                game_special_board[i][j] = 't'
            if i in [1,4,9,12] and j in [1,4,9,12] and (i+3 == j or i == j+3 or i+3 == 13-j or i-3 == 13-j):
                game_special_board[i][j] = '/'
            if i in [2,5,8,11] and j in [2,5,8,11] and (i+3 == j or i == j+3 or i+3 == 13-j or i-3 == 13-j) and i != j and i != 13-j:
                game_special_board[i][j] = '-'
            if i in [1,2,3,4,9,10,11,12] and j in [1,2,3,4,9,10,11,12] and (i == j or i == 13-j):
                game_special_board[i][j] = 'd'
            
    game_special_board[6][6] = '1'
    game_special_board[6][7] = '2'
    game_special_board[7][6] = '3'
    game_special_board[7][7] = '4'
    #print(game_special_board)

    game_board[6][6] = '1'
    game_board[6][7] = '2'
    game_board[7][6] = '3'
    game_board[7][7] = '4'

    #lista_cifre = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,24,25,27,28,30,32,35,36,40,42,45,48,49,50,54,56,60,63,64,70,72,80,81,90]
    #coordonate_board = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N']

    return game_board, game_special_board

In [18]:
game_board, game_special_board = clear_board()

lista_cifre = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,24,25,27,28,30,32,35,36,40,42,45,48,49,50,54,56,60,63,64,70,72,80,81,90]
lista_numere = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,24,25,27,28,30,32,35,36,40,42,45,48,49,50,54,56,60,63,64,70,72,80,81,90]
coordonate_board = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N']

## Extract templates from full train

In [19]:
os.makedirs('templates_full_train/', exist_ok=True)    
for number in lista_numere:
    folder_path = f'templates_full_train/{number}/'
    os.makedirs(folder_path, exist_ok=True)

In [20]:


train_path = "../antrenare/"
output_folder = "./templates_full_train/"  
coordonate_board = "ABCDEFGHIJKLMN"  



def process_patch(patch, output_size=(100, 100)):
    if len(patch.shape) == 3:
        patch = cv.cvtColor(patch, cv.COLOR_BGR2GRAY)

    _, binary_patch = cv.threshold(patch, 90, 255, cv.THRESH_BINARY_INV)

    contours, _ = cv.findContours(binary_patch, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None

    bounding_boxes = [cv.boundingRect(contour) for contour in contours]
    grouped_boxes = group_horizontal_bounding_boxes(bounding_boxes)

    largest_box = calculate_combined_bounding_box(grouped_boxes)

    x, y, w, h = largest_box
    roi = patch[y:y+h, x:x+w]

    center_x, center_y = output_size[1] // 2, output_size[0] // 2

    x_offset = center_x - w // 2
    y_offset = center_y - h // 2

    centered_patch = np.ones((output_size[0], output_size[1]), dtype=np.uint8) * 255

    if x_offset < 0 or y_offset < 0 or x_offset + w > output_size[1] or y_offset + h > output_size[0]:
        pad_top = max(0, -y_offset)
        pad_bottom = max(0, (y_offset + h) - output_size[0])
        pad_left = max(0, -x_offset)
        pad_right = max(0, (x_offset + w) - output_size[1])

        roi = cv.copyMakeBorder(roi, pad_top, pad_bottom, pad_left, pad_right, cv.BORDER_CONSTANT, value=255)

        y_offset = max(0, y_offset)
        x_offset = max(0, x_offset)

    y_end = min(y_offset + roi.shape[0], output_size[0])
    x_end = min(x_offset + roi.shape[1], output_size[1])

    centered_patch[y_offset:y_end, x_offset:x_end] = roi[:y_end-y_offset, :x_end-x_offset]

    return centered_patch


def group_horizontal_bounding_boxes(bounding_boxes, threshold=40):
    bounding_boxes.sort(key=lambda box: box[0])

    groups = []
    current_group = [bounding_boxes[0]]

    for i in range(1, len(bounding_boxes)):
        x1, y1, w1, h1 = bounding_boxes[i-1]
        x2, y2, w2, h2 = bounding_boxes[i]

        if x2 - (x1 + w1) <= threshold and abs(y1 - y2) <= max(h1, h2) * 1.5:
            current_group.append(bounding_boxes[i])
        else:
            groups.append(current_group)
            current_group = [bounding_boxes[i]]

    groups.append(current_group)
    return groups


def calculate_combined_bounding_box(groups):
    largest_group = max(groups, key=len) 

    x_min = min([box[0] for box in largest_group])
    y_min = min([box[1] for box in largest_group])
    x_max = max([box[0] + box[2] for box in largest_group])
    y_max = max([box[1] + box[3] for box in largest_group])

    return x_min, y_min, x_max - x_min, y_max - y_min


def process_file(file, counter_full_train):
    image_path = os.path.join(train_path, file)
    text_name = file.replace(".jpg", ".txt")
    txt_path = os.path.join(train_path, text_name)

    with open(txt_path, 'r') as f:
        line = f.readline().strip()
        position_string, piece_number = line.split()
    print(f"Processing {file}: Position={position_string}, Piece={piece_number}")

    template_image = cv.imread(image_path)
    template_result, template_result_mask, _ = extrage_careu(template_image, 'dark_table_2')
    _, template_result = cv.threshold(template_result, 100, 255, cv.THRESH_BINARY)

    piece_folder = os.path.join(output_folder, piece_number)
    os.makedirs(piece_folder, exist_ok=True)

    
    counter_full_train = extrage_template_cifre_full_train(
        template_image, template_result, lines_horizontal, lines_vertical,
        counter_full_train, position_string, piece_number
    )
    return counter_full_train

def extrage_template_cifre_full_train(img, thresh, lines_horizontal, lines_vertical, counter_full_train, position, number):
    if len(position) == 2:
        lin = int(position[0]) - 1
        col = position[1].upper() 
    elif len(position) == 3:
        lin = int(position[:2]) - 1 
        col = position[2].upper()
    else:
        print(f"Invalid position format: {position}")
        return counter_full_train

    if col in coordonate_board:
        j = coordonate_board.index(col)
    else:
        print(f"Column {col} not found in coordonate_board.")
        return counter_full_train

    i = lin  

    if i < 0 or i >= len(lines_horizontal)-1 or j < 0 or j >= len(lines_vertical)-1:
        print(f"Position out of bounds: {position}")
        return counter_full_train

    for i in range(len(lines_horizontal) - 1):
        for j in range(len(lines_vertical) - 1):
    
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1]
            x_max = lines_horizontal[i + 1][1][1]
            patch = thresh[x_min + 15:x_max - 15, y_min + 15:y_max - 15].copy()
            patch_orig = img[x_min + 15:x_max - 15, y_min + 15:y_max - 15].copy()
            patch_orig = cv.cvtColor(patch_orig, cv.COLOR_BGR2GRAY)
            patch_copy_gray = cv.cvtColor(patch.copy(), cv.COLOR_BGR2GRAY)
    
            medie = np.mean(patch)
            if 120 < medie < 245:
                if i == lin and coordonate_board[j] == col:
                    centered_patch = process_patch(patch_copy_gray, output_size=(100, 100))  # Adjust output_size as needed
                    if centered_patch is not None:
                        patch_path = f"{output_folder}/{number}/{counter_full_train}.jpg"
                        print(f"Saving centered patch: {patch_path}")
                        cv.imwrite(patch_path, centered_patch)
                        counter_full_train += 1
                    else:
                        print(f"Patch at position {position} discarded (mean intensity outside range).")

    return counter_full_train



In [21]:
# # Run 
# counter_full_train = 0
# for file in os.listdir(train_path):
#     if file.endswith(".jpg"):
#         counter_full_train = process_file(file, counter_full_train)


### Add aux templates to fulltrain templates

In [22]:


def extrage_template_cifre_center(img,thresh,lines_horizontal,lines_vertical):
    matrix = np.empty((14,14), dtype='str')
    counter = 0
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1]
            x_max = lines_horizontal[i + 1][1][1]
            patch = thresh[x_min+10:x_max-10, y_min+10:y_max-10].copy()
            patch_orig=img[x_min+10:x_max-10, y_min+10:y_max-10].copy()
            
            patch_orig= cv.cvtColor(patch_orig,cv.COLOR_BGR2GRAY)
            
            patch_copy_gray = cv.cvtColor(patch.copy(), cv.COLOR_BGR2GRAY)
            
            medie = np.mean(patch)
            if medie > 120 and medie < 240:
                matrix[i][j] = 'x'

                centered_patch = process_patch(patch_copy_gray, output_size=(100, 100))
                if centered_patch is not None:
                    #patch_path = f"{output_folder}/{number}/{counter_full_train}.jpg"
                    print(f"templates/{counter}.jpg")
                    cv.imwrite(f"templates/{counter}.jpg", centered_patch)
                    print(f"templates_full_train/{counter}/_{counter}.jpg")
                    cv.imwrite(f"templates_full_train/{counter}/_{counter}.jpg", centered_patch)
                    
                    counter += 1
                    while counter not in lista_cifre and counter < 91:
                        counter +=1
                        
                    #cv.imwrite(patch_path, centered_patch)
                    #counter_full_train += 1
                else:
                    print(f"Patch at position {position} discarded (mean intensity outside range).")
                
                
            else:
                matrix[i][j] = 'o'
    return matrix


In [23]:
# # Run

# template_image = cv.imread(aux_path+'03.jpg')

# template_result,template_result_mask,_ = extrage_careu(template_image, 'dark_table_2', corner_points)
# #show_image('result'+file, template_result)
# _, template_result = cv.threshold(template_result, 120, 255, cv.THRESH_BINARY)
# #show_image('result=thresh'+file, template_result)

# matrice_template = extrage_template_cifre_center(template_image,template_result, lines_horizontal, lines_vertical)
# vizualizare_configuratie(template_result, matrice_template, lines_horizontal, lines_vertical)
# print(matrice_template)
# #show_image('result_diff'+file, template_result)


### Compute mean templates

In [24]:
def medie_templates(templates_path):
    mean_images_dict = {}
    for folder_number in os.listdir(templates_path):
        folder_path = os.path.join(templates_path, folder_number)
        if not os.path.isdir(folder_path):
            continue
        template_images = []
        for file in os.listdir(folder_path):
            if file.endswith('.jpg'):
                file_path = os.path.join(folder_path, file)
                template_image = cv.imread(file_path, cv.IMREAD_GRAYSCALE)
                if template_image is not None:
                    template_images.append(template_image)
        if template_images:
            stacked_images = np.stack(template_images, axis=0)
            mean_image = np.mean(stacked_images, axis=0).astype(np.uint8)
            mean_images_dict[folder_number] = mean_image
    return mean_images_dict


In [25]:
# # Run
# templates_path = "./templates_full_train/"
# mean_images = medie_templates(templates_path)

# templates_mean_path = './templates_mean/'
# os.makedirs(templates_mean_path, exist_ok=True) 
# for piece_number, mean_image in mean_images.items():
#     cv.imwrite(templates_mean_path+f'{piece_number}.jpg', mean_image)


## Number detection and identification functions


In [26]:
def clasifica_cifra(patch):
    matches = [] 
    template_path = './templates_mean/'
    for j in lista_cifre:
        img_template = cv.imread(template_path + str(j) + '.jpg')
        img_template = cv.cvtColor(img_template, cv.COLOR_BGR2GRAY)
        centered_patch = process_patch(patch)
        corr = cv.matchTemplate(centered_patch, img_template, cv.TM_CCOEFF_NORMED)
        corr = np.max(corr)

        matches.append((j, corr))
        if corr > 0.95:
            break
    matches.sort(key=lambda x: x[1], reverse=True)
    #print("Sorted Matches:", matches)

    for digit, score in matches:
        if score > 0.2:  
            return digit

    return matches[0][0]
    

In [27]:
def determina_configuratie_careu_ocifre(img,thresh,lines_horizontal,lines_vertical):
    #print('IMG: ', img)
    #print('THRESH: ', thresh)
    matrix = np.empty((14,14), dtype='int')
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1]
            x_max = lines_horizontal[i + 1][1][1]
            patch = thresh[x_min+10:x_max-10, y_min+10:y_max-10].copy()
            patch_orig=img[x_min+15:x_max-15, y_min+15:y_max-15].copy()
            #print('PATCH_ORIG_',i,'_',j,': ', patch_orig)
            patch_orig= cv.cvtColor(patch_orig,cv.COLOR_BGR2GRAY)
            
            medie = np.mean(patch)
            if medie>120 and medie<245:
              matrix[i][j] = clasifica_cifra(patch_orig)
            else:
              matrix[i][j] = '-1'
    matrix[6][6] = '1'
    matrix[6][7] = '2'
    matrix[7][6] = '3'
    matrix[7][7] = '4'
    return matrix

In [28]:

def determina_configuratie_careu_ocifre_ij(matrix,thresh,lines_horizontal,lines_vertical, lin, col):
    #print('IMG: ', img)
    #print('THRESH: ', thresh)
    #matrix = np.empty((14,14), dtype='int')
    y_min = lines_vertical[col][0][0]
    y_max = lines_vertical[col + 1][1][0]
    x_min = lines_horizontal[lin][0][1]
    x_max = lines_horizontal[lin + 1][1][1]
    #patch = thresh[x_min+10:x_max-10, y_min+10:y_max-10].copy()
    patch_orig=thresh[x_min+15:x_max-15, y_min+15:y_max-15].copy()
            #print('PATCH_ORIG_',i,'_',j,': ', patch_orig)
    patch_orig= cv.cvtColor(patch_orig,cv.COLOR_BGR2GRAY)

    matches = [] 
    template_path = './templates_mean/'
    for j in lista_cifre:
        img_template = cv.imread(template_path + str(j) + '.jpg')
        img_template = cv.cvtColor(img_template, cv.COLOR_BGR2GRAY)
        centered_patch = process_patch(patch_orig)
        corr = cv.matchTemplate(centered_patch, img_template, cv.TM_CCOEFF_NORMED)
        corr = np.max(corr)

        matches.append((j, corr))
        if corr > 0.95:
            break
    
    matches.sort(key=lambda x: x[1], reverse=True)
    #print("Sorted Matches:", matches)

    matrix[lin][col] = matches[0][0]
    
    return matrix, matches
    
#    matrix[i][j], matches = clasifica_cifra(patch_orig)
    
    #return matrix, matches

In [29]:
# img = cv.imread(my_cwd_path+'data/train/01.jpg')
# result=extrage_careu(img)
# show_image('img',result)
# for line in  lines_vertical :
#     cv.line(result, line[0], line[1], (0, 255, 0), 5)
#     for line in  lines_horizontal :
#         cv.line(result, line[0], line[1], (0, 0, 255), 5)
# show_image('img',result)
# _, thresh = cv.threshold(result, 50, 255, cv.THRESH_BINARY_INV)
# matrice = determina_configuratie_careu_ocifre(result, thresh, lines_horizontal, lines_vertical)
# vizualizare_configuratie(result, thresh, lines_horizontal, lines_vertical)
# print(matrice)
# show_image('img'+file, result)

## Test primele 5


In [30]:
# template_files = sorted(os.listdir(f'templates_mean'))
# files = sorted(os.listdir(train_path))
# prev_board = None
# ok_counter = 5
# for file in files:
#   if file[-3:]=='jpg':
#     img = cv.imread(train_path+file)
#     result,result_mask,_ = extrage_careu(img, 'dark_table_2', corner_points)
#     ##show_image('result'+file, result)
#     _, result = cv.threshold(result, 90, 255, cv.THRESH_BINARY)
#     #_, result = cv.threshold(result, 90, 255, cv.THRESH_BINARY)
#     #_, result = cv.threshold(result, 90, 255, cv.THRESH_BINARY)
#     ##show_image('result=thresh'+file, result)

#     matrice_fullboard = determina_configuratie_careu_ox_fullboard(result, lines_horizontal, lines_vertical)
#     vizualizare_configuratie(result, matrice_fullboard, lines_horizontal, lines_vertical)
#     print(matrice_fullboard)
#     ##show_image('result_diff'+file, result)
      
#     matrice_fullboard = determina_configuratie_careu_ocifre(result, result, lines_horizontal, lines_vertical)
#     vizualizare_configuratie(result, matrice_fullboard, lines_horizontal, lines_vertical)
#     print(matrice_fullboard)
#     ##show_image('result_diff'+file, result)
    
#     ok_counter -= 1
    
#     #_, thresh_mask = cv.threshold(result_mask, 50, 255, cv.THRESH_BINARY_INV)
#     #matrice_mask = determina_configuratie_careu_ox(thresh_mask, lines_horizontal, lines_vertical)
#     #vizualizare_configuratie(result_mask, matrice_mask, lines_horizontal, lines_vertical)
#     #print(matrice_mask)
#     #show_image('img-mask'+file, result_mask)
#     if ok_counter < 0:
#         break
#         #pass

## Test predictor vs train, all data

In [31]:
# template_files = sorted(os.listdir(template_path))
# files = sorted(os.listdir(train_path))
# prev_board = None
# prev_matrice_ox = None
# prev_matrice_fullboard = None
# ok_counter = 5
# counter = 0
# lista_greseli = []
# for file in files:
#   if file[-3:]=='jpg':
#     image_path = os.path.join(train_path, file)
#     text_name = file.replace(".jpg", ".txt")
#     txt_path = os.path.join(train_path, text_name)

#     # Read the text file for the piece number and position
#     with open(txt_path, 'r') as f:
#         line = f.readline().strip()
#         position_string, piece_number = line.split()

#     if len(position_string) == 2:
#         lin = int(position_string[0]) - 1 
#         col = position_string[1].upper()  
#     elif len(position_string) == 3:
#         lin = int(position_string[:2]) - 1  
#         col = position_string[2].upper()

#     if file[-6:] == '01.jpg':
#         prev_board = None
#         prev_matrice_ox = None
#         prev_matrice_fullboard = None
      
#     img = cv.imread(image_path)
#     result,result_mask,_ = extrage_careu(img, 'dark_table_2')
#     ##show_image('result'+file, result)
#     _, result = cv.threshold(result, 90, 255, cv.THRESH_BINARY)
#     #_, result = cv.threshold(result, 90, 255, cv.THRESH_BINARY)
#     #_, result = cv.threshold(result, 90, 255, cv.THRESH_BINARY)
#     ##show_image('result=thresh'+file, result)

#     matrice_ox = determina_configuratie_careu_ox_fullboard(result, lines_horizontal, lines_vertical)
#     vizualizare_configuratie(result, matrice_ox, lines_horizontal, lines_vertical)
#     #print(matrice_fullboard)
#     ##show_image('result_diff'+file, result)
    
#     matrice_fullboard = determina_configuratie_careu_ocifre(result, result, lines_horizontal, lines_vertical)
#     vizualizare_configuratie(result, matrice_fullboard, lines_horizontal, lines_vertical)
    
#     #print(matrice_fullboard)
#     ##show_image('result_diff'+file, result)

#     if prev_matrice_ox is not None:
#         coords_elem = np.argwhere(matrice_ox != prev_matrice_ox)
#         i,j = coords_elem[0]
#     else:
#         coords_elem = np.argwhere(matrice_ox == 'x')
#         i,j = coords_elem[0]

#     if int(i)+1 != int(lin)+1 or coordonate_board[j] != col or str(piece_number) != str(matrice_fullboard[i,j]):
#         print(f'Gresit! Imaginea {image_path}, piesa {lin+1}{col} {piece_number} vazuta ca {i+1}{coordonate_board[j]} {matrice_fullboard[i,j]}')
#         lista_greseli.append(image_path)
#     #print(f'Prediction: {i+1}{coordonate_board[j]} {matrice_fullboard[i,j]}' + f'  |  Reality: {lin+1}{col} {piece_number}')
    
#     prev_matrice_ox = matrice_ox
#     prev_matrice_fullboard = matrice_fullboard
    



### Detailed look at the wrong predictions

In [32]:
# for image_path in lista_greseli:
#     img = cv.imread(image_path)
#     result,result_mask,_ = extrage_careu(img, 'dark_table_2')
#     ##show_image('result  '+image_path, result)
#     _, result = cv.threshold(result, 90, 255, cv.THRESH_BINARY)
#     ##show_image('result=thresh  '+image_path, result)

#     matrice_ox = determina_configuratie_careu_ox_fullboard(result, lines_horizontal, lines_vertical)
#     vizualizare_configuratie(result, matrice_ox, lines_horizontal, lines_vertical)
#     ##print(matrice_ox)
#     ##show_image(' '+image_path, result)
    
#     matrice_fullboard = determina_configuratie_careu_ocifre(result, result, lines_horizontal, lines_vertical)
#     vizualizare_configuratie(result, matrice_fullboard, lines_horizontal, lines_vertical)
    
#     ##print(matrice_fullboard)
#     ##show_image(' '+image_path, result)


## Eval()


In [33]:
# def eval(eval_path, game_id, out_path):
#     turns_list = []
#     turns_txt_path = os.path.join(eval_path + f'{game_id}_turns.txt')
    
#     with open(turns_txt_path, 'r') as f:
#         for line in f:
#             line = line.strip()
#             if line:
#                 player_string, round_start = line.split()
#                 turns_list.append((player_string, int(round_start)))
    
#     prev_matrice_ox = None
#     prev_matrice_fullboard = None

#     turns_list.append(('END',51))
#     current_player, current_round_start = turns_list[0]
#     for next_player, next_round_start in turns_list[1:]:
#         for i in range(current_round_start, next_round_start):
#             if i < 10:
#                 image_file = f'{game_id}_0{i}.jpg'
#             else:
#                 image_file = f'{game_id}_{i}.jpg'
#             image_path = os.path.join(eval_path, image_file)
#             img = cv.imread(image_path)
#             result,result_mask,_ = extrage_careu(img, 'dark_table_2')
#             #show_image('result'+image_path, result)
#             _, result = cv.threshold(result, 90, 255, cv.THRESH_BINARY)
#             #show_image('result=thresh'+image_path, result)
            
#             matrice_ox = determina_configuratie_careu_ox_fullboard(result, lines_horizontal, lines_vertical)
#             vizualizare_configuratie(result, matrice_ox, lines_horizontal, lines_vertical)
#             #print(matrice_ox)
#             #show_image('ox '+image_path, result)
            
#             matrice_fullboard = determina_configuratie_careu_ocifre(result, result, lines_horizontal, lines_vertical)
#             vizualizare_configuratie(result, matrice_fullboard, lines_horizontal, lines_vertical)
#             #print(matrice_fullboard)
#             #show_image('fullboard '+image_path, result)
            
#             if prev_matrice_ox is not None:
#                 coords_elem = np.argwhere(matrice_ox != prev_matrice_ox)
#                 lin,col = coords_elem[0]
#             else:
#                 coords_elem = np.argwhere(matrice_ox == 'x')
#                 lin,col = coords_elem[0]

#             prev_matrice_ox = matrice_ox
#             prev_matrice_fullboard = matrice_fullboard
            
#             if i < 10:
#                 with open(out_path + f'{game_id}_0{i}.txt', 'w') as out_file:
#                     out_file.write(f'{lin+1}{coordonate_board[col]} {matrice_fullboard[lin,col]}\n')
#                     #print(f'{lin+1}{coordonate_board[col]} {matrice_fullboard[lin,col]}')
#             else:
#                 with open(out_path + f'{game_id}_{i}.txt', 'w') as out_file:
#                     out_file.write(f'{lin+1}{coordonate_board[col]} {matrice_fullboard[lin,col]}\n')
#                     #print(f'{lin+1}{coordonate_board[col]} {matrice_fullboard[lin,col]}')
#             #print('\n\n')
#         current_player = next_player
#         current_round_start = next_round_start

                



    

In [34]:
# eval_path = '../evaluare/fake_test/'
# out_path = './fake_test_out/'

# os.makedirs(out_path, exist_ok=True)
# eval(eval_path, 1, out_path)

In [35]:
# eval_path = '../antrenare/'
# out_path = './eval_antrenare_3/'

# os.makedirs(out_path, exist_ok=True)
# eval(eval_path, 3, out_path)

In [36]:
# prediction_path = './eval_antrenare_4/'
# truth_path = '../antrenare/'

# files = sorted(os.listdir(prediction_path))
# for file in files:
#     prediction_txt_path = os.path.join(prediction_path, file)
#     truth_txt_path = os.path.join(truth_path, file)
#     with open(prediction_txt_path, 'r') as f:
#         line = f.readline().strip()
#         position_string, piece_number = line.split()
#     if len(position_string) == 2:
#         lin = int(position_string[0]) - 1 
#         col = position_string[1].upper()  
#     elif len(position_string) == 3:
#         lin = int(position_string[:2]) - 1  
#         col = position_string[2].upper()
    
#     with open(truth_txt_path, 'r') as f:
#         line = f.readline().strip()
#         position_string, true_piece_number = line.split()
#     if len(position_string) == 2:
#         true_lin = int(position_string[0]) - 1 
#         true_col = position_string[1].upper()  
#     elif len(position_string) == 3:
#         true_lin = int(position_string[:2]) - 1  
#         true_col = position_string[2].upper()
#     #print(f'Prediction: {lin}{col} {piece_number}  |  Truth: {true_lin}{true_col} {true_piece_number}')

#     if lin!=true_lin or col!=true_col or piece_number!=true_piece_number:
#         print(f'Prediction: {lin}{col} {piece_number}  |  Truth: {true_lin}{true_col} {true_piece_number}')
        

## Eval_score()


In [37]:
def search_piece_score(direction, lin, col, matrice_ox, matrice_fullboard):
    if direction == 'right':
        if col > 11:
            return 0, False
        else:
            if matrice_fullboard[lin,col+1] == '-1' or matrice_fullboard[lin,col+2] == '-1':
                return 0, False
        val1 = matrice_fullboard[lin, col+1]
        val2 = matrice_fullboard[lin, col+2]
        current_val = matrice_fullboard[lin, col]
    elif direction == 'left':
        if col < 2:
            return 0, False
        else:
            if matrice_fullboard[lin,col-1] == '-1' or matrice_fullboard[lin,col-2] == '-1':
                return 0, False
        val1 = matrice_fullboard[lin, col-1]
        val2 = matrice_fullboard[lin, col-2]
        current_val = matrice_fullboard[lin, col]
    elif direction == 'up':
        if lin < 2:
            return 0, False
        else:
            if matrice_fullboard[lin-1,col] == '-1' or matrice_fullboard[lin-2,col] == '-1':
                return 0, False
        val1 = matrice_fullboard[lin-1, col]
        val2 = matrice_fullboard[lin-2, col]
        current_val = matrice_fullboard[lin, col]
    elif direction == 'down':
        if lin > 11:
            return 0, False
        else:
            if matrice_fullboard[lin+1,col] == '-1' or matrice_fullboard[lin+2,col] == '-1':
                return 0, False
        val1 = matrice_fullboard[lin+1, col]
        val2 = matrice_fullboard[lin+2, col]
        current_val = matrice_fullboard[lin, col]
    
    if val1 == -1 or val2 == -1:
        return 0, False
            
    if game_special_board[lin,col] == '+':
        if current_val == val1 + val2:
            return current_val, True
    elif game_special_board[lin,col] == '-':
        if current_val == val1 - val2 or current_val == val2 - val1:
            return current_val, True
    elif game_special_board[lin,col] == '*':
        if current_val == val1 * val2:
            return current_val, True
    elif game_special_board[lin,col] == '/':
        if ( val2!=0 and current_val == val1 / val2) or (val1!=0 and current_val == val2 / val1):
            return current_val, True
    elif game_special_board[lin,col] == 'd':
        if current_val == val1 + val2:
            return current_val*2, True
        elif current_val == val1 - val2 or current_val == val2 - val1:
            return current_val*2, True
        elif current_val == val1 * val2:
            return current_val*2, True
        elif (val2!=0 and current_val == val1 / val2) or (val1!=0 and current_val == val2 / val1):
            return current_val*2, True
    elif game_special_board[lin,col] == 't':
        if current_val == val1 + val2:
            return current_val*3, True
        elif current_val == val1 - val2 or current_val == val2 - val1:
            return current_val*3, True
        elif current_val == val1 * val2:
            return current_val*3, True
        elif (val2!=0 and current_val == val1 / val2) or (val1!=0 and current_val == val2 / val1):
            return current_val*3, True
    elif game_special_board[lin,col] == '#':
        if current_val == val1 + val2:
            return current_val, True
        elif current_val == val1 - val2 or current_val == val2 - val1:
            return current_val, True
        elif current_val == val1 * val2:
            return current_val, True
        elif (val2!=0 and current_val == val1 / val2) or (val1!=0 and current_val == val2 / val1):
            return current_val, True

    return 0, False



In [38]:
def eval_score(eval_path, game_id, out_path):
    turns_list = []
    turns_txt_path = os.path.join(eval_path + f'{game_id}_turns.txt')
    
    with open(turns_txt_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line:
                player_string, round_start = line.split()
                turns_list.append((player_string, int(round_start)))
    
    prev_matrice_ox = None
    prev_matrice_fullboard = None
    
    turns_list.append(('END',51))
    current_player, current_round_start = turns_list[0]

    current_game_board, _ = clear_board()
    
    for next_player, next_round_start in turns_list[1:]:
        player_score = 0
        
        for i in range(current_round_start, next_round_start):
            piece_score = 0
            
            if i < 10:
                image_file = f'{game_id}_0{i}.jpg'
            else:
                image_file = f'{game_id}_{i}.jpg'
            image_path = os.path.join(eval_path, image_file)
            img = cv.imread(image_path)
            result,result_mask,_ = extrage_careu(img, 'dark_table_2')
            #show_image('result'+image_path, result)
            _, result = cv.threshold(result, 90, 255, cv.THRESH_BINARY)
            #show_image('result=thresh'+image_path, result)
            
            matrice_ox = determina_configuratie_careu_ox_fullboard(result, lines_horizontal, lines_vertical)
            vizualizare_configuratie(result, matrice_ox, lines_horizontal, lines_vertical)
            #print(matrice_ox)
            #show_image('ox '+image_path, result)
            
            if prev_matrice_ox is not None:
                coords_elem = np.argwhere(matrice_ox != prev_matrice_ox)
                lin,col = coords_elem[0]
            else:
                coords_elem = np.argwhere(matrice_ox == 'x')
                lin,col = coords_elem[0]

            matrice_fullboard = determina_configuratie_careu_ocifre(result, result, lines_horizontal, lines_vertical)
            vizualizare_configuratie(result, matrice_fullboard, lines_horizontal, lines_vertical)
            #print(matrice_fullboard)
            #show_image('fullboard '+image_path, result)
            
            current_game_board, template_matches = determina_configuratie_careu_ocifre_ij(current_game_board, result, lines_horizontal, lines_vertical, lin, col)
            vizualizare_configuratie(result, current_game_board, lines_horizontal, lines_vertical)
            #print(current_game_board)
            
            #piece_score += search_piece_score('right', lin, col, matrice_ox, current_game_board)
            #piece_score += search_piece_score('left', lin, col, matrice_ox, current_game_board)
            #piece_score += search_piece_score('up', lin, col, matrice_ox, current_game_board)
            #piece_score += search_piece_score('down', lin, col, matrice_ox, current_game_board)
            #piece_score = int(piece_score)
            try:
                cnt = 0
                found = False
                while piece_score == 0 and found is False:
                    current_game_board[lin,col] = template_matches[cnt][0]
                    local_score, found_local = search_piece_score('right', lin, col, matrice_ox, current_game_board)
                    piece_score += local_score
                    found = found or found_local
                    local_score, found_local = search_piece_score('left', lin, col, matrice_ox, current_game_board)
                    piece_score += local_score
                    found = found or found_local
                    local_score, found_local = search_piece_score('up', lin, col, matrice_ox, current_game_board)
                    piece_score += local_score
                    found = found or found_local
                    local_score, found_local = search_piece_score('down', lin, col, matrice_ox, current_game_board)
                    piece_score += local_score
                    found = found or found_local
                    piece_score = int(piece_score)
                    cnt += 1
                    
            except IndexError:
                print('Except: no match satisfies ecuation, likely break in chain')
                current_game_board[lin,col] = template_matches[0][0]
                anomaly_score = 0
                for ii in range(0,14):
                    for jj in range(0,14):
                        if int(current_game_board[ii,jj]) != int(matrice_fullboard[ii,jj]):
                            print(f'Anomaly found at {ii,jj}')
                            found = False
                            local_score, found = search_piece_score('right', ii, jj, matrice_ox, current_game_board)
                            anomaly_score -= local_score
                            local_score, found = search_piece_score('left', ii, jj, matrice_ox, current_game_board)
                            anomaly_score -= local_score
                            local_score, found = search_piece_score('up', ii, jj, matrice_ox, current_game_board)
                            anomaly_score -= local_score
                            local_score, found = search_piece_score('down', ii, jj, matrice_ox, current_game_board)
                            anomaly_score -= local_score
                            #if found is True:
                            anomaly_score = int(anomaly_score)
                            
                            found = False
                            local_score, found = search_piece_score('right', ii, jj, matrice_ox, matrice_fullboard)
                            anomaly_score += local_score
                            local_score, found = search_piece_score('left', ii, jj, matrice_ox, matrice_fullboard)
                            anomaly_score += local_score
                            local_score, found = search_piece_score('up', ii, jj, matrice_ox, matrice_fullboard)
                            anomaly_score += local_score
                            local_score, found = search_piece_score('down', ii, jj, matrice_ox, matrice_fullboard)
                            anomaly_score += local_score
                            anomaly_score = int(anomaly_score)
                            if found is True:
                                piece_score += anomaly_score 
                                current_game_board[ii,jj] = matrice_fullboard[ii][jj]
                                print(f'Corrected anomaly to {matrice_fullboard[ii][jj]}')
                                local_score, found = search_piece_score('right', lin, col, matrice_ox, current_game_board)
                                piece_score += local_score
                                local_score, found = search_piece_score('left', lin, col, matrice_ox, current_game_board)
                                piece_score += local_score
                                local_score, found = search_piece_score('up', lin, col, matrice_ox, current_game_board)
                                piece_score += local_score
                                local_score, found = search_piece_score('down', lin, col, matrice_ox, current_game_board)
                                piece_score += local_score
                                piece_score = int(piece_score)
                                
                                for anomaly_file_name in os.listdir(out_path):
                                    if anomaly_file_name.endswith('scores.txt') or anomaly_file_name.endswith('turns.txt'): 
                                        continue
                                    if (not anomaly_file_name.startswith(f'{game_id}_')) or (not anomaly_file_name.endswith(f'.txt')):
                                        continue
                                    anomaly_file_path = os.path.join(out_path, anomaly_file_name)
        
                                    with open(anomaly_file_path, 'r+') as anomaly_file:
                                        anomaly_line = anomaly_file.readline().strip()
                                        #print(f'Anomaly line: {anomaly_line}')
                                        anomaly_position_string, anomaly_value = anomaly_line.split()

                                        if len(anomaly_position_string) == 2:
                                            anomaly_lin = int(anomaly_position_string[0])
                                            anomaly_col = anomaly_position_string[1].upper()  
                                        elif len(anomaly_position_string) == 3:
                                            anomaly_lin = int(anomaly_position_string[:2])  
                                            anomaly_col = anomaly_position_string[2].upper()
                                            
                                        if anomaly_lin == ii+1 and anomaly_col == coordonate_board[jj]:
                                            anomaly_file.seek(0)
                                            anomaly_file.write(f'{anomaly_lin}{anomaly_col} {int(current_game_board[ii,jj])}')
                                            anomaly_file.truncate()
                                            print(f'Corrected anomaly file {anomaly_file_name}: {anomaly_lin}{anomaly_col} {int(current_game_board[ii,jj])}')
                                            
                                
                            ###### TODO list of scores and revert to that position 
                            ###### TODO edit the out file
                            
                
            player_score += piece_score
            
            prev_matrice_ox = matrice_ox
            prev_matrice_fullboard = matrice_fullboard
            
            if i < 10:
                with open(out_path + f'{game_id}_0{i}.txt', 'w') as out_file:
                    out_file.write(f'{lin+1}{coordonate_board[col]} {int(current_game_board[lin,col])}\n')
                    print(f'{lin+1}{coordonate_board[col]} {int(current_game_board[lin,col])}  |  Score {piece_score}')
            else:
                with open(out_path + f'{game_id}_{i}.txt', 'w') as out_file:
                    out_file.write(f'{lin+1}{coordonate_board[col]} {int(current_game_board[lin,col])}\n')
                    print(f'{lin+1}{coordonate_board[col]} {int(current_game_board[lin,col])}  |  Score {piece_score}')
            #print('\n\n')
            
        print(f'{current_player} {current_round_start} {player_score}')
        with open(out_path + f'{game_id}_scores.txt', 'a') as out_file:
            out_file.write(f'{current_player} {current_round_start} {player_score}\n')
                    
        current_player = next_player
        current_round_start = next_round_start

                



    

In [39]:
# print(game_board)
# print(game_special_board)

### Run eval_score() on ./train

In [40]:
# eval_path = '../antrenare/'
# out_path = './eval_antrenare_2/'

# os.makedirs(out_path, exist_ok=True)
# eval_score(eval_path, 2, out_path)

In [41]:
# eval_path = '../antrenare/'
# out_path = './eval_antrenare_3/'

# os.makedirs(out_path, exist_ok=True)
# eval_score(eval_path, 3, out_path)

In [42]:
# eval_path = '../antrenare/'
# out_path = './eval_antrenare_1/'

# os.makedirs(out_path, exist_ok=True)
# eval_score(eval_path, 1, out_path)

In [43]:
# eval_path = '../antrenare/'
# out_path = './eval_antrenare_4/'

# os.makedirs(out_path, exist_ok=True)
# eval_score(eval_path, 4, out_path)

In [44]:
# game_id = 3
# prediction_path = f'./eval_antrenare_{game_id}/'
# truth_path = '../antrenare/'

# files = sorted(os.listdir(prediction_path))
# for file in files:
#     if file == f'{game_id}_scores.txt':
#         continue
#     prediction_txt_path = os.path.join(prediction_path, file)
#     truth_txt_path = os.path.join(truth_path, file)
#     with open(prediction_txt_path, 'r') as f:
#         line = f.readline().strip()
#         position_string, piece_number = line.split()
#     if len(position_string) == 2:
#         lin = int(position_string[0]) - 1 
#         col = position_string[1].upper()  
#     elif len(position_string) == 3:
#         lin = int(position_string[:2]) - 1  
#         col = position_string[2].upper()
    
#     with open(truth_txt_path, 'r') as f:
#         line = f.readline().strip()
#         position_string, true_piece_number = line.split()
#     if len(position_string) == 2:
#         true_lin = int(position_string[0]) - 1 
#         true_col = position_string[1].upper()  
#     elif len(position_string) == 3:
#         true_lin = int(position_string[:2]) - 1  
#         true_col = position_string[2].upper()
#     #print(f'Prediction: {lin}{col} {piece_number}  |  Truth: {true_lin}{true_col} {true_piece_number}')

#     if lin!=true_lin or col!=true_col or piece_number!=true_piece_number:
#         print(f'Prediction: {lin}{col} {piece_number}  |  Truth: {true_lin}{true_col} {true_piece_number}')
        

In [45]:
# game_id = 1
# eval_path = '../evaluare/fake_test/'
# out_path = f'./fake_test_out_{game_id}/'

# os.makedirs(out_path, exist_ok=True)
# eval_score(eval_path, game_id, out_path)

In [46]:
# game_id = 1
# prediction_path = f'./fake_test_out_{game_id}/'
# truth_path = '../evaluare/cod_evaluare/fake_test_gt/'

# files = sorted(os.listdir(prediction_path))
# for file in files:
#     if file == f'{game_id}_scores.txt':
#         continue
#     prediction_txt_path = os.path.join(prediction_path, file)
#     truth_txt_path = os.path.join(truth_path, file)
#     with open(prediction_txt_path, 'r') as f:
#         line = f.readline().strip()
#         position_string, piece_number = line.split()
#     if len(position_string) == 2:
#         lin = int(position_string[0]) - 1 
#         col = position_string[1].upper()  
#     elif len(position_string) == 3:
#         lin = int(position_string[:2]) - 1  
#         col = position_string[2].upper()
    
#     with open(truth_txt_path, 'r') as f:
#         line = f.readline().strip()
#         position_string, true_piece_number = line.split()
#     if len(position_string) == 2:
#         true_lin = int(position_string[0]) - 1 
#         true_col = position_string[1].upper()  
#     elif len(position_string) == 3:
#         true_lin = int(position_string[:2]) - 1  
#         true_col = position_string[2].upper()
#     #print(f'Prediction: {lin}{col} {piece_number}  |  Truth: {true_lin}{true_col} {true_piece_number}')

#     if lin!=true_lin or col!=true_col or piece_number!=true_piece_number:
#         print(f'Prediction: {lin}{col} {piece_number}  |  Truth: {true_lin}{true_col} {true_piece_number}')
        

### Cod testare oficial pe fake test

In [47]:
# game_id = 1
# eval_path = '../evaluare/fake_test/'
# out_path = f'../evaluare/fisiere_solutie/364_Alexandru_Marius-Critian/'

# os.makedirs(out_path, exist_ok=True)
# eval_score(eval_path, game_id, out_path)

In [48]:
# def compare_annotations_position_token(filename_predicted,filename_gt,verbose=0):
# 	p = open(filename_predicted,"rt")  	
# 	gt = open(filename_gt,"rt") 

	
# 	line_gt = gt.readline()
# 	current_pos_gt, current_token_gt = line_gt.split()
# 	if verbose:
# 		print(current_pos_gt,current_token_gt)
	
# 	line_p = p.readline()
# 	current_pos_p, current_token_p = line_p.split()
# 	if verbose:
# 		print(current_pos_p,current_token_p)

	
# 	match_positions = 1
# 	match_tokens = 1
# 	if(current_pos_p != current_pos_gt):
# 		match_positions = 0
# 	if(current_token_p != current_token_gt):
# 		match_tokens = 0
	

# 	points_positions = 0.025 * match_positions
# 	points_tokens = 0.01 * match_tokens	


# 	return points_positions, points_tokens

# #change this on your machine pointing to your results (txt files)
# #predictions_path_root = "../fisiere_solutie/331_Alexe_Bogdan/"
# #predictions_path_root = "../evaluare/fisiere_solutie/331_Alexe_Bogdan/"
# #predictions_path_root = '../evaluare/fisiere_solutie/364_Alexandru_Marius-Critian/'
# predictions_path_root = './fake_test_out_1/'

# #change this on your machine to point to the ground-truth test
# gt_path_root = "../evaluare/cod_evaluare/fake_test_gt/"



# def compare_annotations_score(filename_predicted,filename_gt,verbose=0):
# 	p = open(filename_predicted,"rt")  	
# 	gt = open(filename_gt,"rt") 

# 	all_lines_gt = gt.readlines()
# 	all_lines_p = p.readlines()


# 	number_lines_gt = len(all_lines_gt)

# 	points_scores = 0

# 	try:
# 		for i in range(0,number_lines_gt):
# 			line_gt = all_lines_gt[i]
# 			line_p = all_lines_p[i]
# 			if (line_gt==line_p):
# 				points_scores += 0.04
# 	except:
# 		pass

# 	return points_scores


# #change this to 1 if you want to print results at each turn
# verbose = 1
# total_points = 0
# for game in range(1,2):
# # change this for game in range(1,5):
# 	points_score = 0
# 	for turn in range(1,51):
		
# 		name_turn = str(turn)
# 		if(turn < 10):
# 			name_turn = '0'+str(turn)

# 		filename_predicted = predictions_path_root + str(game) + '_' + name_turn + '.txt'
# 		filename_gt = gt_path_root + str(game) + '_' + name_turn + '.txt'

# 		game_turn = str(game) + '_' + name_turn
# 		points_position = 0
# 		points_tokens = 0	

# 		try:
# 			points_position, points_tokens  = compare_annotations_position_token(filename_predicted,filename_gt,verbose)
# 		except:
# 			print("Pentru imaginea: ", game_turn, " functia de evaluare a intampinat o eroare")

# 		print("Imaginea: ", game_turn, "Puncte task 1: ", points_position, "Puncte task 2: ",points_tokens)
		
# 		total_points = total_points + points_position + points_tokens

# 	filename_predicted_scores = predictions_path_root + str(game) + '_scores' + '.txt'
# 	filename_gt_scores = gt_path_root + str(game) + '_scores' + '.txt'
# 	points_score = compare_annotations_score(filename_predicted_scores,filename_gt_scores,verbose=0)
# 	print("Puncte task 3 ", points_score)

# 	total_points = total_points + points_score
# 	print("Total puncte = ", total_points)


# print(total_points)

## Run eval_score() pe ./test

In [50]:
for game_id in range(1,5):
    eval_path = './testare/'
    out_path = f'./463_Alexandru_Marius-Critian/'

    os.makedirs(out_path, exist_ok=True)
    eval_score(eval_path, game_id, out_path)

9G 2  |  Score 2
9H 6  |  Score 6
8I 7  |  Score 7
7I 1  |  Score 1
9I 8  |  Score 16
Player1 1 32
10I 15  |  Score 15
9J 14  |  Score 14
8J 11  |  Score 11
7J 2  |  Score 2
Player2 6 42
6J 13  |  Score 13
10J 25  |  Score 50
10K 40  |  Score 40
Player1 10 103
11I 7  |  Score 7
10H 24  |  Score 24
10G 5  |  Score 5
11H 30  |  Score 30
11G 10  |  Score 10
Player2 13 76
11F 20  |  Score 20
9F 12  |  Score 12
11E 2  |  Score 2
8F 7  |  Score 7
Player1 18 41
7F 19  |  Score 19
11D 18  |  Score 36
11C 9  |  Score 9
Player2 22 64
11B 9  |  Score 9
11A 81  |  Score 81
6G 4  |  Score 4
12H 6  |  Score 6
13H 36  |  Score 36
Player1 25 136
14H 42  |  Score 126
12G 2  |  Score 2
12I 8  |  Score 8
13I 1  |  Score 1
Player2 30 137
13G 35  |  Score 35
Except: no match satisfies ecuation, likely break in chain
Anomaly found at (10, 5)
Anomaly found at (10, 7)
Anomaly found at (11, 11)
Anomaly found at (12, 12)
Anomaly found at (13, 6)
12L 64  |  Score 0
Except: no match satisfies ecuation, likely bre

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\templmatch.cpp:588: error: (-215:Assertion failed) corr.rows <= img.rows + templ.rows - 1 && corr.cols <= img.cols + templ.cols - 1 in function 'cv::crossCorr'


In [ ]:
# def compare_annotations_position_token(filename_predicted,filename_gt,verbose=0):
# 	p = open(filename_predicted,"rt")  	
# 	gt = open(filename_gt,"rt") 

	
# 	line_gt = gt.readline()
# 	current_pos_gt, current_token_gt = line_gt.split()
# 	if verbose:
# 		print(current_pos_gt,current_token_gt)
	
# 	line_p = p.readline()
# 	current_pos_p, current_token_p = line_p.split()
# 	if verbose:
# 		print(current_pos_p,current_token_p)

	
# 	match_positions = 1
# 	match_tokens = 1
# 	if(current_pos_p != current_pos_gt):
# 		match_positions = 0
# 	if(current_token_p != current_token_gt):
# 		match_tokens = 0
	

# 	points_positions = 0.025 * match_positions
# 	points_tokens = 0.01 * match_tokens	


# 	return points_positions, points_tokens

# #change this on your machine pointing to your results (txt files)
# #predictions_path_root = "../fisiere_solutie/331_Alexe_Bogdan/"
# #predictions_path_root = "../evaluare/fisiere_solutie/331_Alexe_Bogdan/"
# predictions_path_root = './463_Alexandru_Marius-Critian/'

# #change this on your machine to point to the ground-truth test
# gt_path_root = "./testare_gt/"



# def compare_annotations_score(filename_predicted,filename_gt,verbose=0):
# 	p = open(filename_predicted,"rt")  	
# 	gt = open(filename_gt,"rt") 

# 	all_lines_gt = gt.readlines()
# 	all_lines_p = p.readlines()


# 	number_lines_gt = len(all_lines_gt)

# 	points_scores = 0

# 	try:
# 		for i in range(0,number_lines_gt):
# 			line_gt = all_lines_gt[i]
# 			line_p = all_lines_p[i]
# 			if (line_gt==line_p):
# 				points_scores += 0.04
# 	except:
# 		pass

# 	return points_scores


# #change this to 1 if you want to print results at each turn
# verbose = 1
# total_points = 0
# for game in range(1,5):
# # change this for game in range(1,5):
# 	points_score = 0
# 	for turn in range(1,51):
		
# 		name_turn = str(turn)
# 		if(turn < 10):
# 			name_turn = '0'+str(turn)

# 		filename_predicted = predictions_path_root + str(game) + '_' + name_turn + '.txt'
# 		filename_gt = gt_path_root + str(game) + '_' + name_turn + '.txt'

# 		game_turn = str(game) + '_' + name_turn
# 		points_position = 0
# 		points_tokens = 0	

# 		try:
# 			points_position, points_tokens  = compare_annotations_position_token(filename_predicted,filename_gt,verbose=0)
# 		except:
# 			print("Pentru imaginea: ", game_turn, " functia de evaluare a intampinat o eroare")

# 		print("Imaginea: ", game_turn, "Puncte task 1: ", points_position, "Puncte task 2: ",points_tokens)
		
# 		total_points = total_points + points_position + points_tokens

# 	filename_predicted_scores = predictions_path_root + str(game) + '_scores' + '.txt'
# 	filename_gt_scores = gt_path_root + str(game) + '_scores' + '.txt'
# 	points_score = compare_annotations_score(filename_predicted_scores,filename_gt_scores,verbose=1)
# 	print("Puncte task 3 ", points_score)

# 	total_points = total_points + points_score
# 	print("Total puncte = ", total_points)


# print(total_points)